In [1]:
include("../src/RadialBasisFunction.jl")
include("../src/DiscreteAdjoint.jl")
include("../src/GradientDescentTools.jl")

using LinearAlgebra, Zygote, Plots
using .RadialBasisFunction
using .DiscreteAdjoint
using .GradientDescentTools

const dir_for_figures = "../results/figures/";


## Adjoint method applied to RBF approximation

In this notebook, the parameter that we want to optimize, $b$, is geometric; so in this case we have $J(u,b)$ instead of $J(u, q)$  
Note that each time $b$ is updated the shape of the object change

In [24]:
a, b = 0, 2π
N = 40

# Points will change each time b will change
X = range(a,b, length=N)
boundary_idxs = [1, N]
X_boundary = X[boundary_idxs]
internal_idxs = 2:(N-1)
X_internal = X[internal_idxs]

q(x) = -sin.(x)  # Known in advance (param)
boundary_conditions = [0, 0]  # Decided by us

φ(r) = r .^ 3  # r = |x-x_i| ≥ 0
ddφ(r) = 6*r   #second derivative d²/dx²

n_points_in_stencil = 7

C = global_matrix(X, φ, ddφ, n_points_in_stencil)
u_approximated = solve_RBF(C, X, internal_idxs, boundary_idxs, boundary_conditions, q);

In [25]:
# Defininig J (obj function)
E = 2.0
J(u_num, b) = 1/2 * ( b/(N-1) * sum(u_num) - E)^2  #* u_num is so small as if it does not exist

# Initial values
b_k = b
u_k = u_approximated
push!(u_k, boundary_conditions[end])
pushfirst!(u_k, boundary_conditions[1]);

# Define all the functions that will be used to compute dJdb
φ(x_i,x_j) = abs(x_i - x_j) ^ 3
dφdb(i,j) = 1/(N-1) * abs(i - j) ^ 3        # dφdb(i,j) = d/db [φ(xi(b),xj(b))]
Δφ(x_i,x_j) = 6 * abs(x_i - x_j)            # Δφ(x,xj) = d²/dx² (φ(x,xj))
dΔφdb(i,j) = 6 * 1/(N-1) * abs(i - j)       # dΔφdb(i,j) = d/db [d²/dx² (φ(x,xj))]
;

In [26]:
# The adjoint+AD procedure

∂J∂u, ∂J∂b = gradient((u, b) -> J(u,b), u_k[internal_idxs], b_k)
λ = C[internal_idxs, internal_idxs]' \ ∂J∂u
dCdb = zeros(size(C))

#* It would be possible to cycle only on X_internal, but different indices would be needed
for i ∈ eachindex(X)

    # Since optimimzation is carried out only on internal points --> skip boundary points
    if i ∈ boundary_idxs
        continue
    end


    # Find the associated stencil's points
    xi = X[i]
    neighbours_idxs = nearest_neighbour_search(xi, X, n_points_in_stencil)
    setdiff!(neighbours_idxs, boundary_idxs)


    # Compute what is needed to compute ∂ci∂b
    u_on_stencil = u_k[neighbours_idxs]

    ci_of_strencil = C[i, neighbours_idxs]

    dΨidb = dΔφdb.(fill(i, size(neighbours_idxs)), neighbours_idxs)

    Φi = [φ(xi, xj) for xi in X[neighbours_idxs], xj in X[neighbours_idxs]]
    Mi = Φi

    dΦidb = [dφdb(i,j) for i in neighbours_idxs, j in neighbours_idxs]
    dMidb = dΦidb


    # Compute ∂ci∂b and fill the related matrix
    dci_of_stencil_db = Mi' * (dΨidb - dMidb'*ci_of_strencil)
    dCdb[i, neighbours_idxs] = dci_of_stencil_db

end

dJdb = ∂J∂b' - λ' * dCdb[internal_idxs, internal_idxs] * u_k[internal_idxs]


2.051461849397822e-11

In [28]:
h = 1e-6

b_h = b + h

# If b is changed by h --> also u changes (thus you need to solve again the RBF problem)
X_h = range(a,b_h, length=N)
X_boundary_h = X[boundary_idxs]
X_internal_h = X[internal_idxs]

# Needs to be redefined with r since were redefined outside
φ(r) = r .^ 3  # r = |x-x_i| ≥ 0
ddφ(r) = 6*r   #second derivative d²/dx² (because the differential problem deals with u" = d²u/dx²)

# Get the changed u as result of the changed b
C_h = global_matrix(X_h, φ, ddφ, n_points_in_stencil)
u_h = solve_RBF(C_h, X, internal_idxs, boundary_idxs, boundary_conditions, q);

# Compute the finite difference
dJdb = (J(u_h, b_h) - J(u_k, b)) / h

4.973799150320701e-8